In [1]:
import os
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices=false" # 关闭部分 XLA 行为
os.environ["TF_CUDNN_DETERMINISTIC"] = "1" # 强制确定性算法，减少搜索

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import KFold
import itertools


2025-12-15 11:27:34.615751: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 11:27:34.699790: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-15 11:27:36.201572: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


#### Data Preparation and Preprocessing
load and normalize images

In [3]:
import tensorflow as tf

# Define dataset paths
train_dir = "/home/zyh/Fruit-Classifier/data/train_augment"    # directory with subfolders for each class (train set)


IMG_SIZE = 224   # image size (can be adjusted as needed)



model structure

In [4]:
from tensorflow.keras import layers, models, regularizers

def create_cnn_model():

    model = models.Sequential()

    #Convolution blocks
    model.add(layers.Conv2D(32, kernel_size = (3,3), 
                    padding='same',
                    input_shape=(IMG_SIZE, IMG_SIZE, 3),
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2)) 

    model.add(layers.Conv2D(64, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2)) 

    model.add(layers.Conv2D(128, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))
    model.add(layers.Conv2D(256, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))
    model.add(layers.Conv2D(512, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))

    model.add(layers.Conv2D(1024, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))

    model.add(layers.Conv2D(2048, kernel_size = (3,3), 
                    padding='same',
                    activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(4,activation='softmax'))


    #model.summary()

    
    
    return model

In [5]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Wrap the model creation in KerasClassifier for use in GridSearchCV
def build_model(learning_rate=0.01, optimizer_name='Adam', activation='relu', 
                dropout_rate=0.2, l2_rate=0.0):
    # Build the CNN model with given hyperparams
    model = create_cnn_model()
    # Choose optimizer
    if optimizer_name == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:  # 'SGD'
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'], jit_compile=False)
    return model


# 全内存数据加载函数 (速度优化的关键)
def load_data_to_memory(data_dir, img_size):
    print(f"正在加载数据到内存: {data_dir} ...")
    X = []
    y = []
    # 确保类名顺序一致: ['apple', 'banana', 'mixed', 'orange']
    classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    print(f"检测到的类别: {classes}")
    
    for label_idx, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        files = os.listdir(class_dir)
        for fname in files:
            if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                fpath = os.path.join(class_dir, fname)
                # 加载并调整大小
                img = load_img(fpath, target_size=(img_size, img_size))
                # 转为数组并归一化 (0-1)
                img_array = img_to_array(img) / 255.0
                
                X.append(img_array)
                y.append(label_idx)
                
    X = np.array(X)
    # 将标签转为 One-hot 编码 (e.g., [0, 1, 0, 0])
    y = tf.keras.utils.to_categorical(np.array(y), num_classes=len(classes))
    print(f"数据加载完成! X shape: {X.shape}, y shape: {y.shape}")
    return X, y



# 一次性读取所有数据到 RAM (320张图约占用 150MB~200MB 内存，非常安全)
X_all, y_all = load_data_to_memory(train_dir, IMG_SIZE)


#  极速版 Grid Search
param_grid = {
    'learning_rate': [0.001, 0.0005,0.0001],
    'optimizer_name': [ 'Adam'],
    'batch_size': [32],
    'activation': ['relu'],
    'dropout_rate': [0.2,0.4],
    'l2_rate': [0.0, 0.001]
}


combinations = list(itertools.product(*param_grid.values()))
total_runs = len(combinations)
keys = list(param_grid.keys())

# Storage for results
experiment_results = []
best_loss = float('inf') # Finding Minimum
best_params = {}

print(f"\n🚀 Starting Grid Search (Target: Minimum Validation Loss)")
print(f"Total combinations: {total_runs} (with 5-Fold CV)\n")

# Use 5-Fold as requested
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for run_idx, combo in enumerate(combinations):
    params = dict(zip(keys, combo))
    # Extract batch_size
    build_args = {k: v for k, v in params.items() if k != 'batch_size'}
    current_batch_size = params['batch_size']
    
    fold_losses = []
    fold_accs = []
    
    print(f"👉 [{run_idx+1}/{total_runs}] Testing: {params} ...", end="", flush=True)
    
    for train_idx, val_idx in kfold.split(X_all):
        tf.keras.backend.clear_session() # Clear GPU memory
        
        X_train, X_val = X_all[train_idx], X_all[val_idx]
        y_train, y_val = y_all[train_idx], y_all[val_idx]
        
        model = build_model(**build_args)
        
        # Training
        model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            batch_size=current_batch_size,
            epochs=15, 
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=False, monitor='val_loss')],
            verbose=0 # Silent mode for speed
        )
        
        # Evaluation
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
        fold_losses.append(val_loss)
        fold_accs.append(val_acc)
        


    # Calculate Statistics
    mean_loss = np.mean(fold_losses)
    std_loss = np.std(fold_losses)
    mean_acc = np.mean(fold_accs)
    
    print(f" -> Loss: {mean_loss:.4f} (±{std_loss:.4f})")
    
    # Store results
    res_entry = params.copy()
    res_entry['mean_loss'] = mean_loss
    res_entry['std_loss'] = std_loss
    res_entry['mean_acc'] = mean_acc
    # Create short label for plotting
    res_entry['Label'] = f"LR={params['learning_rate']}\nBS={params['batch_size']}\nDrop={params['dropout_rate']}"
    experiment_results.append(res_entry)

    # Check for best (Minimum Loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        best_params = params
        print(f"   🔥 New Best Model found! (Low Loss: {best_loss:.4f})")

print("\n================ Search Complete ================")
print(f"Best Loss: {best_loss:.4f}")
print(f"Best Params: {best_params}")


# 4. Visualization 
if len(experiment_results) > 0:
    df_results = pd.DataFrame(experiment_results)
    
    # Sort by mean loss for better visualization
    df_results = df_results.sort_values(by='mean_loss', ascending=True)
    
    plt.figure(figsize=(16, 8))
    
    # Bar plot for Mean Loss
    sns.barplot(x='Label', y='mean_loss', data=df_results, palette='viridis', ci=None)
    
    # Add Error Bars (Standard Deviation)
    plt.errorbar(x=range(len(df_results)), 
                 y=df_results['mean_loss'], 
                 yerr=df_results['std_loss'], 
                 fmt='none', c='black', capsize=5, label='Stability (Std Dev)')
    
    plt.title('Hyperparameter Performance: Mean Validation Loss (Lower is Better)', fontsize=14)
    plt.ylabel('Mean K-Fold Loss', fontsize=14)
    plt.xlabel('Parameter Combinations', fontsize=14)
    plt.xticks(rotation=45,ha='right', fontsize=10)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print("\n Chart Guide:")
    print("1. Height of bar: Average Loss (Lower is better).")
    print("2. Black line (Error Bar): Stability. Shorter lines mean the model is more stable across different data splits.")

正在加载数据到内存: /home/zyh/Fruit-Classifier/data/train_augment ...
检测到的类别: ['apple', 'banana', 'mixed', 'orange']
数据加载完成! X shape: (308, 224, 224, 3), y shape: (308, 4)

🚀 Starting Grid Search (Target: Minimum Validation Loss)
Total combinations: 12 (with 5-Fold CV)

👉 [1/12] Testing: {'learning_rate': 0.001, 'optimizer_name': 'Adam', 'batch_size': 32, 'activation': 'relu', 'dropout_rate': 0.2, 'l2_rate': 0.0} ...

/home/zyh/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1765769259.888632 1619173 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7537 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6
2025-12-15 11:27:43.702267: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91600


 -> Loss: 0.9052 (±0.2793)
   🔥 New Best Model found! (Low Loss: 0.9052)
👉 [2/12] Testing: {'learning_rate': 0.001, 'optimizer_name': 'Adam', 'batch_size': 32, 'activation': 'relu', 'dropout_rate': 0.2, 'l2_rate': 0.001} ... -> Loss: 0.8359 (±0.3367)
   🔥 New Best Model found! (Low Loss: 0.8359)
👉 [3/12] Testing: {'learning_rate': 0.001, 'optimizer_name': 'Adam', 'batch_size': 32, 'activation': 'relu', 'dropout_rate': 0.4, 'l2_rate': 0.0} ... -> Loss: 0.8408 (±0.4512)
👉 [4/12] Testing: {'learning_rate': 0.001, 'optimizer_name': 'Adam', 'batch_size': 32, 'activation': 'relu', 'dropout_rate': 0.4, 'l2_rate': 0.001} ...

2025-12-15 11:30:41.024317: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 72.00MiB (rounded to 75497472)requested by op StatefulPartitionedCall/adam/Sqrt_25
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-12-15 11:30:41.024444: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1049] BFCAllocator dump for GPU_0_bfc
2025-12-15 11:30:41.024468: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1056] Bin (256): 	Total Chunks: 673, Chunks in use: 666. 168.2KiB allocated for chunks. 166.5KiB in use in bin. 47.1KiB client-requested in use in bin.
2025-12-15 11:30:41.024485: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1056] Bin (512): 	Total Chunks: 122, Chunks in use: 119. 66.2KiB allocated for chunks. 64.5KiB in use in bin. 59.5KiB cli

ResourceExhaustedError: Graph execution error:

Detected at node adam/Sqrt_25 defined at (most recent call last):
  File "/home/zyh/anaconda3/envs/tf/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/zyh/anaconda3/envs/tf/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/zyh/anaconda3/envs/tf/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/zyh/anaconda3/envs/tf/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/zyh/anaconda3/envs/tf/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 758, in start

  File "/home/zyh/anaconda3/envs/tf/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/home/zyh/anaconda3/envs/tf/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/home/zyh/anaconda3/envs/tf/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/home/zyh/anaconda3/envs/tf/

A few notes on the above:

We defined build_model to accept the hyperparams. If activation='leaky_relu', inside create_cnn_model we would handle that by setting layers with no activation and adding LeakyReLU layers. (This implementation detail can be handled with an if inside create_cnn_model.)


------------------------------------------------------

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping
# early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# grid_search.fit(X_train, y_train, **{'callbacks': [early_stop], 'validation_split': 0.1})


-------------------------------------------------------

#### Model Training with Best Hyperparameters and Early Stopping

In [ ]:
# # Build final model with best hyperparams
# best_model = create_cnn_model(activation=best_params['activation'], 
#                               dropout_rate=best_params['dropout_rate'], 
#                               l2_rate=best_params['l2_rate'])
# if best_params['optimizer_name'] == 'Adam':
#     final_optimizer = tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'])
# else:
#     final_optimizer = tf.keras.optimizers.SGD(learning_rate=best_params['learning_rate'])
# best_model.compile(optimizer=final_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# # Setup callbacks for early stopping (and optional learning rate reduction, checkpoints as in original code)
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# callbacks = [
#     EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
#     ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1),
#     ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
# ]

# # Train the model with early stopping
# history = best_model.fit(train_generator, epochs=50,  # start with an upper bound, early stopping will likely stop earlier
#                          validation_data=val_generator, 
#                          callbacks=callbacks, verbose=1)


Results and Visualization of Training Progress

In [ ]:
# val_loss, val_acc = best_model.evaluate(val_generator)
# print(f"Validation Accuracy: {val_acc:.2%}, Validation Loss: {val_loss:.4f}")


In [ ]:
# import matplotlib.pyplot as plt

# # Extract history data
# epochs = range(1, len(history.history['loss'])+1)
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']
# train_acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']

# # Plot Loss Curves
# plt.figure(figsize=(6,4))
# plt.plot(epochs, train_loss, 'bo-', label='Training Loss')
# plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
# plt.title('Training vs Validation Loss')
# plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.show()

# # Plot Accuracy Curves
# plt.figure(figsize=(6,4))
# plt.plot(epochs, train_acc, 'bo-', label='Training Accuracy')
# plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
# plt.title('Training vs Validation Accuracy')
# plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.show()
